# **Análisis de distancias semánticas entre términos GO de Proteínas Moonlighting**
Este notebook explora las distancias semánticas entre términos GO asociados con proteínas moonlighting. Calcularemos el promedio y el máximo de las distancias semánticas entre pares de términos GO dentro de las categorías de Cellular Component (CC), Molecular Function (MF) y Biological Process (BP) para cada proteína.


## Montaje de Google Drive
Montamos Google Drive para acceder y guardar archivos directamente desde este entorno.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Preparación del entorno de análisis
Cargamos el conjunto de datos de proteínas moonlighting para realizar nuestro análisis.

In [ ]:
import pandas as pd

moonlights_metrics = pd.read_csv('/content/drive/My Drive/moon_metrics_v2.csv')
moonlights_metrics.columns = moonlights_metrics.columns.str.strip()

In [ ]:
# Visualizamos las columnas del dataset.
moonlights_metrics.columns

Index(['proteinentry', 'accessioncode', 'organism', 'go_term1', 'go_category1',
       'go_description1', 'evidence1', 'go_term2', 'go_category2',
       'go_description2', 'evidence2', 'resnikdistance',
       'minimumbranchlength'],
      dtype='object')

## Definición de funciones para el cálculo de estadísticas
Definimos una función que calculará el promedio y el máximo de la distancia semántica entre términos GO. Además, esta función identificará los pares de términos GO que presentan la mayor distancia semántica dentro de cada categoría para cada proteína.


In [ ]:
def calculate_stats_and_unique_pairs(group):
    avg_mbl = group['minimumbranchlength'].mean()
    max_mbl = group['minimumbranchlength'].max()

    # Identificamos y extraemos los pares de términos con distancia máxima, evitando duplicados.
    max_pairs = group[group['minimumbranchlength'] == max_mbl][['go_term1', 'go_term2']].drop_duplicates()
    max_pairs_list = max_pairs.apply(lambda x: (x['go_term1'], x['go_term2']), axis=1).tolist()

    return pd.Series({
        'Avg_MBL': avg_mbl,
        'Max_MBL': max_mbl,
        'GO_Pairs_Max_MBL': max_pairs_list
    })

## Cálculo y combinación de estadísticas por categoría
Agrupamos los datos por 'accesioncode' (UniProt ID) y por categoría de término GO para calcular las estadísticas de distancia semántica.


In [ ]:
results_category1 = moonlights_metrics.groupby(['accessioncode', 'go_category1']).apply(calculate_stats_and_unique_pairs).rename_axis(['UniProt IDs', 'Category']).reset_index()
results_category2 = moonlights_metrics.groupby(['accessioncode', 'go_category2']).apply(calculate_stats_and_unique_pairs).rename_axis(['Uniprot IDs', 'Category']).reset_index()

# Combinamos ambos resultados obtenidos utilizando pd.concat.
results = pd.concat([results_category1, results_category2])

## Consolidación de pares y cálculo de estadísticas finales
Unificamos los pares de términos GO para evitar duplicados y calculamos estadísticas consolidadas para cada proteína y cada categoría de término GO.


In [ ]:
def consolidate_pairs(pairs_list):
    unique_pairs = set()
    for pairs in pairs_list:
        unique_pairs.update(pairs)
    return list(unique_pairs)

final_results = results.groupby(['UniProt IDs', 'Category']).agg({
    'Avg_MBL': 'mean',
    'Max_MBL': 'max',
    'GO_Pairs_Max_MBL': lambda x: consolidate_pairs(x)
}).reset_index()

# Visualizamos los resultados finales consolidados para cada categoría y proteína.
print(final_results)

     UniProt IDs Category    Avg_MBL  Max_MBL  \
0     A0A0H3LHU4        C   2.000000        2   
1     A0A0H3LHU4        F   8.071429       14   
2     A0A1D8PFS4        C   6.666667        9   
3     A0A4Y1WAR4        C   2.000000        2   
4     A0A4Y1WAR4        F  10.000000       10   
...          ...      ...        ...      ...   
1426      Q9Z0E3        C   4.600000        7   
1427      Q9Z0E3        P  13.285714       19   
1428      Q9ZVD5        C   6.000000        9   
1429      Q9ZVD5        F   8.166667       12   
1430      Q9ZVD5        P  14.250000       21   

                                       GO_Pairs_Max_MBL  
0                            [(GO:0005576, GO:0005829)]  
1     [(GO:0004356, GO:0005525), (GO:0004356, GO:001...  
2                            [(GO:0005777, GO:0062040)]  
3     [(GO:0005615, GO:0009986), (GO:0005829, GO:000...  
4                            [(GO:0002020, GO:0004365)]  
...                                                 ...  
1426 

## Filtrado y visualización de resultados por categoría
Filtramos y visualizamos los resultados finales para cada categoría de término GO (CC, MF, BP).

In [ ]:
moon_results_C = final_results[final_results['Category'] == 'C']
moon_results_F = final_results[final_results['Category'] == 'F']
moon_results_P = final_results[final_results['Category'] == 'P']

# Visualizamos los resultados filtrados por categorías.
print("Resultados para Categoría C:\n", moon_results_C)
print("\nResultados para Categoría F:\n", moon_results_F)
print("\nResultados para Categoría P:\n", moon_results_P)

Resultados para Categoría C:
      UniProt IDs Category   Avg_MBL  Max_MBL  \
0     A0A0H3LHU4        C  2.000000        2   
2     A0A1D8PFS4        C  6.666667        9   
3     A0A4Y1WAR4        C  2.000000        2   
6         A5D8V6        C  9.000000        9   
9         B9PW35        C  2.000000        2   
...          ...      ...       ...      ...   
1420      Q9Y6Q9        C  4.066667        8   
1423      Q9Y6X0        C  2.000000        2   
1424      Q9Y7F0        C  4.600000        8   
1426      Q9Z0E3        C  4.600000        7   
1428      Q9ZVD5        C  6.000000        9   

                                       GO_Pairs_Max_MBL  
0                            [(GO:0005576, GO:0005829)]  
2                            [(GO:0005777, GO:0062040)]  
3     [(GO:0005615, GO:0009986), (GO:0005829, GO:000...  
6                            [(GO:0000813, GO:0010008)]  
9                            [(GO:0005615, GO:0090651)]  
...                                          

## Preparación del índice y combinación de resultados
Preparamos un índice usando `moon_dataset.csv`, que incluye todas las proteínas moonlighting bajo estudio. Añadimos columnas específicas que almacenan la distancia semántica máxima entre pares de términos GO para las categorías CC y MF, así como los pares que presentan estas distancias máximas.


In [ ]:
# Cargamos nuestro dataset.
moon_dataset = pd.read_csv("/content/drive/My Drive/moon_dataset.csv")

# Combinamos los DataFrames results_C y results_F con moon_dataset utilizando la columna 'UniProt IDs',
# e incorporamos las distancias mínimas de rama máximas y los pares de términos GO correspondientes.
moon_dataset.set_index('UniProt IDs', inplace=True)
moon_results_C.set_index('UniProt IDs', inplace=True)
moon_results_F.set_index('UniProt IDs', inplace=True)

final_dataset = moon_dataset.join([
    moon_results_C[['Max_MBL', 'GO_Pairs_Max_MBL']].rename(columns={
        'Max_MBL': 'Max_MBL_CC',
        'GO_Pairs_Max_MBL': 'CC_Pairs_Max_MBL'
    }),
    moon_results_F[['Max_MBL', 'GO_Pairs_Max_MBL']].rename(columns={
        'Max_MBL': 'Max_MBL_MF',
        'GO_Pairs_Max_MBL': 'MF_Pairs_Max_MBL'
    })
], how='left')  # Combinamos utilizando 'left' para mantener todas las entradas de moon_dataset.

# Asignamos un valor predeterminado de 0 a las columnas de distancia máxima donde no existen datos.
final_dataset['Max_MBL_CC'].fillna(0, inplace=True)
final_dataset['Max_MBL_MF'].fillna(0, inplace=True)

# Asignamos un valor predeterminado 'No data' a las columnas que detallan los pares de términos GO
# cuando no existen datos disponibles.
final_dataset['CC_Pairs_Max_MBL'].fillna('No data', inplace=True)
final_dataset['MF_Pairs_Max_MBL'].fillna('No data', inplace=True)

# Reseteamos el índice para convertir 'UniProt IDs' en una columna regular del DataFrame.
final_dataset.reset_index(inplace=True)

# Visualizamos los resultados obtenidos.
print(final_dataset)

    UniProt IDs                                             PDB ID  \
0        Q9CW03                                         2WD5; 7DG5   
1        Q43155                                                NaN   
2        P30041                                   1PRX; 5B6M; 5B6N   
3        P06745  1U0E; 1U0F; 1U0G; 2CVP; 2CXN; 2CXO; 2CXP; 2CXQ...   
4        P10809  4PJ1; 6HT7; 6MRC; 6MRD; 7AZP; 7L7S; 8G7J; 8G7K...   
..          ...                                                ...   
764      Q22289                                                NaN   
765      O17670                                                NaN   
766      P55853                                               5XQM   
767      Q9VCD5                                                NaN   
768      P48601                                                NaN   

                                             Gene Name  \
0    {'Name': 'Smc3', 'Synonyms': ['Bam', 'Bmh', 'C...   
1    {'Name': 'FdGOGAT {ECO:0000305}', 'ORF

## Comparación y filtrado de distancias semánticas máximas. Reordenación del conjunto de datos.

Determinamos cuál de las distancias semánticas máximas (CC o MF) es mayor para cada proteína y almacenamos este valor mayor en una nueva columna 'Max_Semantic_Distance'. Posteriormente, eliminamos las entradas que no contienen información relevante y ordenamos los resultados para facilitar su interpretación.

In [ ]:
final_dataset['Highest_MBL'] = final_dataset[['Max_MBL_CC', 'Max_MBL_MF']].max(axis=1)

# Definimos las condiciones para identificar y eliminar filas que no contienen datos relevantes.
condiciones = (
    (final_dataset['Max_MBL_CC'] == 0) &
    (final_dataset['CC_Pairs_Max_MBL'] == 'No data') &
    (final_dataset['Max_MBL_MF'] == 0) &
    (final_dataset['MF_Pairs_Max_MBL'] == 'No data')
)

# Filtramos el DataFrame para eliminar las filas que cumplen todas las condiciones especificadas anteriormente.
final_dataset_filtered = final_dataset[~condiciones]

# Ordenamos los resultados de manera ascendente según el valor de 'Max_Semantic_Distance'.
sorted_final_dataset = final_dataset_filtered.sort_values('Highest_MBL')

# Visualizamos los resultados obtenidos.
print(sorted_final_dataset)

    UniProt IDs                                             PDB ID  \
402      P09169                                               1I78   
601      Q9Y2X8                                                NaN   
519      Q05086  1C4Z; 1D5F; 1EQX; 2KR1; 4GIZ; 4XR8; 6SJV; 6SLM...   
526      Q9Y6X0                                                NaN   
758      Q8BH75                                               2OGB   
..          ...                                                ...   
757      E9Q401  2MC2; 3IM5; 3IM6; 3IM7; 3QR5; 4ETV; 4KEI; 4KEJ...   
194      P68187  1Q12; 1Q1B; 1Q1E; 2AWN; 2AWO; 2R6G; 3FH6; 3GD7...   
606      Q09472  1L3E; 1P4Q; 2K8F; 2MH0; 2MZD; 3BIY; 3I3J; 3IO2...   
239      P13569  1XMI; 1XMJ; 2BBO; 2BBS; 2BBT; 2LOB; 2PZE; 2PZF...   
138      P15336                       1BHI; 1T2K; 4H36; 6ZQS; 6ZR5   

                                             Gene Name  \
402  {'Name': 'ompT', 'OrderedLocusNames': ['b0565'...   
601         {'Name': 'UBE2D4', 'Synonyms': 

## Etiquetado y almacenamiento de los resultados finales
Añadimos una nueva columna 'Class', asignando valor 'True' a todas las entradas para reflejar que todas las proteínas del conjunto de datos son multifuncionales. Finalmente, guardamos los resultados en un archivo CSV, que será objeto de futuros análisis.


In [ ]:
# Añadimos la columna 'Class'.
sorted_final_dataset['Class'] = True

# Guardamos el DataFrame resultante.
sorted_final_dataset.to_csv('/content/drive/My Drive/moon_stats_dataset.csv', index=False)

# Visualizamos los primeros registros del nuevo DataFrame.
print(sorted_final_dataset.head())

    UniProt IDs                                             PDB ID  \
402      P09169                                               1I78   
601      Q9Y2X8                                                NaN   
519      Q05086  1C4Z; 1D5F; 1EQX; 2KR1; 4GIZ; 4XR8; 6SJV; 6SLM...   
526      Q9Y6X0                                                NaN   
758      Q8BH75                                               2OGB   

                                             Gene Name  \
402  {'Name': 'ompT', 'OrderedLocusNames': ['b0565'...   
601         {'Name': 'UBE2D4', 'Synonyms': ['UBCH5D']}   
519  {'Name': 'UBE3A {ECO:0000312|HGNC:HGNC:12496}'...   
526       {'Name': 'SETBP1', 'Synonyms': ['KIAA0437']}   
758   {'Name': 'Rnf41', 'Synonyms': ['Flrf', 'Nrdp1']}   

                           Protein Name  \
402                          Protease 7   
601  Ubiquitin-conjugating enzyme E2 D4   
519        Ubiquitin-protein ligase E3A   
526                 SET-binding protein   
758   E3 ubiqui